In [1]:
import xarray as xr
import xroms
import numpy as np
import matplotlib.pyplot as plt
import cartopy
import cmocean.cm as cmo
import pandas as pd

# How to calculate with `xroms`

Here we demonstrate a number of calculations built into `xroms`, through accessors to `DataArrays` and `Datasets`.

### Load in data

More information at in [load_data notebook](load_data.ipynb)

In [2]:
loc = 'http://barataria.tamu.edu:8080/thredds/dodsC/forecast_latest/txla2_his_f_latest.nc'
chunks = {'ocean_time':1}
ds = xroms.open_netcdf(loc, chunks=chunks)  # also adds coordinates and sets up grid in ds.grid

## Set up

### `xarray` Datasets

Use `xarray` accessor in `xroms` to easily perform these calculations with syntax 

    ds.xroms.[method]
    
Importantly, the `xroms` accessor to a `Dataset` is initialized with an `xgcm` grid object, stored at `ds.xroms.grid`, which is used to perform the basic grid calculations.

Generally the built-in calculations take in the horizontal then vertical grid label you want the calculation to be on:

    ds.xroms.dudz('rho', 's_rho')  # to make sure result is on rho horizontal grid and s_rho vertical grid
    
or

    ds.xroms.dudz()  # return on whatever grid it is calculated on
    
These are not stored as properties in the accessor since they depend on the user's choice of grids, but since they are lazily evaluated, they are quick to rerun as needed.

### `xarray` DataArrays

A few of the more basic methods in `xroms` are available to `DataArrays` too. Unlike a `Dataset` in `xroms`, a `DataArray` does not know about its grid (a `Dataset` has its grid information stored in `ds.xroms.grid`). So, the built-in `xroms` methods for `DataArrays` require the grid object to be input.

    ds.temp.xroms.to_grid(ds.grid, hcoord='psi', scoord='s_w')

## Basic functionality

### Change grids

A ROMS user frequently needs to move between horizontal and vertical grids, so it is built into many of the function wrappers, but you can also do it as a separate function. It can also be done to both `Datasets` and `DataArrays` with slightly different syntax. Here we change salinity from its default grids to be on the psi grid horizontally and the s_w grid vertically:

In [3]:
ds.xroms.to_grid('salt', 'psi', 's_w');   # Dataset
ds.salt.xroms.to_grid(ds.xroms.grid, 'psi', 's_w');   # DataArray

You can also go to the original `xroms` function and avoid the `xarray` accessor if you prefer, though the point of the accessor approach (that is, `ds.xroms...`) is to be easier to remember and less code to write generally.

In [5]:
xroms.to_grid(ds.salt, ds.xroms.grid, 'psi', 's_w');

## Calculations

### Vertical derivative, d/dz

Take a vertical derivative of a variable:

In [6]:
ds.xroms.ddz('salt');  # Dataset

In [7]:
ds.salt.xroms.ddz(ds.xroms.grid);  # DataArray

In [8]:
xroms.ddz(ds.salt, ds.xroms.grid);  # No accessor

A convenience function `calc_ddz` is also available to wrap `ddz` along with `to_grid` so that calculating a vertical derivative and changing grids can occur in one line:

In [9]:
ds.xroms.calc_ddz('salt', 'dsaltdz', hcoord='psi', scoord='s_rho', sboundary='extend', sfill_value=np.nan);  # Dataset

In [10]:
ds.salt.xroms.calc_ddz(ds.xroms.grid, 'dsaltdz', hcoord='psi', scoord='s_rho', sboundary='extend', sfill_value=np.nan);  # DataArray

In [11]:
xroms.calc_ddz(ds.salt, ds.xroms.grid, 'dsaltdz', hcoord='psi', scoord='s_rho', sboundary='extend', sfill_value=np.nan);  # No accessor

### Vertical shear

Since it is a common use case, there are specific methods to return the u and v components of vertical shear on their own grids, as follows, and note that as always these functions will take in `hcoord` and `scoord` to shift grids. These are just available for Datasets.

In [13]:
ds.xroms.dudz();  

In [14]:
ds.xroms.dvdz();

If we want to calculate something with both, we need them on the same grid. For this, we can input the desired resultant grid:

In [15]:
ds.xroms.dudz(hcoord='rho', scoord='s_rho')**2 + ds.xroms.dvdz(hcoord='rho', scoord='s_rho')**2

,Array,Chunk
Bytes,2.95 GB,28.32 MB
Shape,"(96, 30, 191, 671)","(1, 28, 189, 669)"
Count,48787 Tasks,2592 Chunks
Type,float64,numpy.ndarray


### density

In [3]:
ds.xroms.get_rho()

<xarray.DataArray (ocean_time: 96, s_rho: 30, eta_rho: 191, xi_rho: 671)>
dask.array<truediv, shape=(96, 30, 191, 671), dtype=float64, chunksize=(1, 30, 191, 671), chunktype=numpy.ndarray>
Coordinates:
  * s_rho       (s_rho) float64 -0.9833 -0.95 -0.9167 ... -0.05 -0.01667
    lon_rho     (eta_rho, xi_rho) float64 dask.array<chunksize=(191, 671), meta=np.ndarray>
    lat_rho     (eta_rho, xi_rho) float64 dask.array<chunksize=(191, 671), meta=np.ndarray>
  * ocean_time  (ocean_time) datetime64[ns] 2020-08-25T01:00:00 ... 2020-08-29
    z_rho       (ocean_time, s_rho, eta_rho, xi_rho) float64 dask.array<chunksize=(1, 30, 191, 671), meta=np.ndarray>
    z_rho0      (s_rho, eta_rho, xi_rho) float64 dask.array<chunksize=(30, 191, 671), meta=np.ndarray>
Dimensions without coordinates: eta_rho, xi_rho

### buoyancy frequency

In [3]:
ds.xroms.N2(hcoord=None, scoord='s_w', sboundary='fill', sfill_value=np.nan)

,Array,Chunk
Bytes,3.05 GB,27.68 MB
Shape,"(96, 31, 191, 671)","(1, 27, 191, 671)"
Count,20766 Tasks,480 Chunks
Type,float64,numpy.ndarray


### vertical vorticity

In [4]:
ds.xroms.vort()

,Array,Chunk
Bytes,3.03 GB,29.53 MB
Shape,"(96, 31, 190, 670)","(1, 29, 190, 670)"
Count,31533 Tasks,288 Chunks
Type,float64,numpy.ndarray


### ertel

In [5]:
ds.xroms.ertel()

<xarray.DataArray (ocean_time: 96, s_rho: 30, eta_rho: 191, xi_rho: 671)>
dask.array<add, shape=(96, 30, 191, 671), dtype=float64, chunksize=(1, 28, 189, 669), chunktype=numpy.ndarray>
Coordinates:
  * ocean_time  (ocean_time) datetime64[ns] 2020-08-25T01:00:00 ... 2020-08-29
  * s_rho       (s_rho) float64 -0.9833 -0.95 -0.9167 ... -0.05 -0.01667
    lon_rho     (eta_rho, xi_rho) float64 dask.array<chunksize=(191, 671), meta=np.ndarray>
    lat_rho     (eta_rho, xi_rho) float64 dask.array<chunksize=(191, 671), meta=np.ndarray>
Dimensions without coordinates: eta_rho, xi_rho